## 0. Pre-processing of inut data to allow correct selection of field names etc.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib
from shapely.geometry import Polygon
%matplotlib inline

pd.set_option("display.max_columns", 101)

In [ ]:
test_data_name='test_data3'

test_data_path='../'+test_data_name+'/'

os.chdir(test_data_path)
%run -i "m2l_config.py"


In [ ]:
bbox=(minx,miny,maxx,maxy)
bbox2=str(minx)+","+str(miny)+","+str(maxx)+","+str(maxy)
lat_point_list = [miny, miny, maxy, maxy, maxy]
lon_point_list = [minx, maxx, maxx, minx, minx]
bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom]) 

geology = gpd.read_file(geology_file,bbox=bbox)
lines=gpd.read_file(fault_file,bbox=bbox)
structures = gpd.read_file(structure_file,bbox=bbox)

base=geology.plot(column=c_l['c'],figsize=(10,10),edgecolor='#000000',linewidth=0.2)
structures.plot(ax=base, color='none',edgecolor='black')
lines.plot(ax=base,cmap='rainbow',column=c_l['f'],figsize=(10,10),linewidth=0.4)
polygon.plot(ax=base, color='none',edgecolor='black')





In [ ]:
display(geology.head(10))

In [ ]:
display(lines.head(10))

In [ ]:
display(structures.head(10))

In [ ]:
import geopandas as gpd
import os
from shapely.geometry import LineString, Point
from math import fabs
from map2loop import m2l_utils
import numpy as np
from shapely import geometry, ops


path_faults='../test_data3/tmp/faults_clip.shp'
def merge_faults(path_faults,c_l):
    eps=0.1
    faults_clip=gpd.read_file(path_faults)
    linelist=[]
    pairlist=[]
    fulllist=[]
    for indx,aline in faults_clip.iterrows():
        if(c_l['fault'] in aline[c_l['f']]):
            flt_ls=LineString(aline.geometry)
            linelist+=(flt_ls.coords[0][0],flt_ls.coords[0][1],flt_ls.coords[1][0],flt_ls.coords[1][1],flt_ls.coords[len(flt_ls.coords)-2][0],flt_ls.coords[len(flt_ls.coords)-2][1],flt_ls.coords[len(flt_ls.coords)-1][0],flt_ls.coords[len(flt_ls.coords)-1][1],aline[c_l['o']])    #display(linelist)
            fulllist+=(aline[c_l['o']],flt_ls)
            
    step=2
    fl2 = [fulllist[i:i+step] for i in range(0, len(fulllist), step)]

    step=9
    ll2 = [linelist[i:i+step] for i in range(0, len(linelist), step)]
    
    for a in range(0,len(faults_clip)):
        for b in range(a,len(faults_clip)):
            if(fabs(ll2[a][0]-ll2[b][0])<eps and fabs(ll2[a][1]-ll2[b][1])<eps and not ll2[a][8] == ll2[b][8]): 
                if(m2l_utils.tri_angle(ll2[a][0],ll2[a][1],ll2[a][2],ll2[a][3],ll2[b][2],ll2[b][3])>170):
                    pairlist+=(ll2[a][8],ll2[b][8],0)

            if(fabs(ll2[a][0]-ll2[b][6])<eps and fabs(ll2[a][1]-ll2[b][7])<eps and not ll2[a][8] == ll2[b][8]): 
                if(m2l_utils.tri_angle(ll2[a][0],ll2[a][1],ll2[a][2],ll2[a][3],ll2[b][4],ll2[b][5])>170):
                    pairlist+=(ll2[a][8],ll2[b][8],1)
            if(fabs(ll2[a][6]-ll2[b][6])<eps and fabs(ll2[a][7]-ll2[b][7])<eps and not ll2[a][8] == ll2[b][8]):
                if(m2l_utils.tri_angle(ll2[a][6],ll2[a][7],ll2[a][4],ll2[a][5],ll2[b][4],ll2[b][5])>170):
                    pairlist+=(ll2[a][8],ll2[b][8],0)
    step=3        
    pl2 = [pairlist[i:i+step] for i in range(0, len(pairlist), step)]
      
    display(fl2)
    
merge_faults(path_faults,c_l)    

In [ ]:
    #for pts in range (0,len(fl2[0][1].coords)):
    #    print(fl2[0][0],fl2[0][1].coords[pts])

    step=9
    ll2 = [linelist[i:i+step] for i in range(0, len(linelist), step)]
    
    for a in range(0,len(faults_clip)):
        for b in range(0,len(faults_clip)):
            if(fabs(ll2[a][0]-ll2[b][0])<eps and fabs(ll2[a][1]-ll2[b][1])<eps and not ll2[a][8] == ll2[b][8]): 
                if(m2l_utils.tri_angle(ll2[a][0],ll2[a][1],ll2[a][2],ll2[a][3],ll2[b][2],ll2[b][3])>170):
                    pairlist+=(ll2[a][8],ll2[b][8],0)

            if(fabs(ll2[a][0]-ll2[b][6])<eps and fabs(ll2[a][1]-ll2[b][7])<eps and not ll2[a][8] == ll2[b][8]): 
                if(m2l_utils.tri_angle(ll2[a][0],ll2[a][1],ll2[a][2],ll2[a][3],ll2[b][4],ll2[b][5])>170):
                    pairlist+=(ll2[a][8],ll2[b][8],1)
            if(fabs(ll2[a][6]-ll2[b][6])<eps and fabs(ll2[a][7]-ll2[b][7])<eps and not ll2[a][8] == ll2[b][8]):
                if(m2l_utils.tri_angle(ll2[a][6],ll2[a][7],ll2[a][4],ll2[a][5],ll2[b][4],ll2[b][5])>170):
                    pairlist+=(ll2[a][8],ll2[b][8],0)
    step=3        
    pl2 = [pairlist[i:i+step] for i in range(0, len(pairlist), step)]
      
    display(pl2)
    marray=np.zeros((len(pl2)))
    nmult=0
    
    for apair in range (0,len(pl2)-1):
        for bpair in range (apair+1,len(pl2)):
            if(pl2[bpair][0] == pl2[apair][0] or pl2[bpair][0] == pl2[apair][1]):
                print('***',pl2[bpair][0])
                marray[nmult]=pl2[bpair][0]
                nmult=nmult+1
            if(pl2[bpair][1] == pl2[apair][0] or pl2[bpair][1] == pl2[apair][1]):
                print('***',bpair,pl2[bpair][1])
                marray[nmult]=pl2[bpair][1]
                nmult=nmult+1
    display(nmult,marray)            
    return(0)


# combine them into a multi-linestring
#multi_line = geometry.MultiLineString([line_a, line_b, line_c])
#print(multi_line)  # prints MULTILINESTRING ((0 0, 1 1), (1 1, 2 2), (2 2, 3 3))

# you can now merge the lines
#merged_line = ops.linemerge(multi_line)
#print(merged_line)  # prints LINESTRING (0 0, 1 1, 2 2, 3 3)

    for pairs in range (0,len(pl2)):
        print(pl2[pairs][0],pl2[pairs][1],pl2[pairs][2])
        rowa=faults_clip.loc[faults_clip[c_l['o']] == pl2[pairs][0]]
        rowb=faults_clip.loc[faults_clip[c_l['o']] == pl2[pairs][1]]
        for pts in rowb.geometry:
            flt_lsb=LineString(pts)
            for pts in range (0,len(flt_lsb.coords)):
                print(flt_lsb.coords[pts])
        for pts in rowa.geometry:
            flt_lsa=LineString(pts)
            for pts in range (1,len(flt_lsa.coords)):
                print(flt_lsa.coords[pts])



merge_faults(path_faults,c_l)

In [ ]:
from shapely.geometry import LineString
line = LineString([(0, 0), (0, 1), (1, 1), (1, 1)])
line.area

line.length

In [ ]:
#merge faults

#load faults 
#if not matching endpoints:
    #copy faults to new clean geodataframe 
#else:
    #copy faults to new dirty database and add column to geodataframe
#loop unit no more dirty
    #if ends match:
        #append merged faults copy name if available and flag two sources as used
#combine clean geodataframe and newly cleaned geodataframe       
#save out new faults as shapefile 


#save array of pairs ids and processed flag

#